In [630]:
import pandas as pd
import pandas_ta
import ta

In [631]:
df = pd.read_csv('../BTC_ETH_1week_1m.csv')

In [632]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std*nstd
    lower_band = sma - std*nstd
    return upper_band, lower_band

ta.volatility.AverageTrueRange
symbols = ['BTC']


nstd = 2

for symbol in symbols:
    df[f'{symbol}_SMA'] = sma(df[f'{symbol}-USD_Open'], 20)

    df[f'{symbol}_Upper_Band'], df[f'{symbol}_Lower_Band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_SMA'], 20, nstd)
    
    df[f'{symbol}_ATR'] = ta.volatility.average_true_range(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], window=14)
    
    dfs = pandas_ta.supertrend(high=df[f'{symbol}-USD_High'], low=df[f'{symbol}-USD_Low'], close=df[f'{symbol}-USD_Close'], period=10, length=None, multiplier=3)
    df[f'{symbol}_SuperDir'] = dfs['SUPERTd_7_3.0']

    df[f'{symbol}_RSI'] = ta.momentum.rsi(close=df[f'{symbol}-USD_Close'], window=20)

df.dropna(inplace=True)

In [633]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, symbols):
        self.short_unit = None
        self.short_amount = 0
        self.short_price = 0
        self.long_price = 0

        self.sl = 0
        self.max_inner_buy = .20
        self.band_stop_run = 0.0    # <= 1
        self.slm = 7



        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.symbols = symbols
        
        self.buys = []
        self.sells = []
        
        self.wins = 0
        self.losses = 0
        self.ties = 0
        
        self.trading_fee_multiplier = 0.9996      # VIP level 0, paying fees with BNB 0.99975
        self.number_of_buys = len(self.buys)

        self.cross_up = {}
        self.cross_down = {}
        self.sma_cross_up = {}
        self.sma_cross_down = {}
        self.reset_crosses([symbols])

    def buy_long(self, symbol, long_price, atr, time):
        self.balance_amount = (self.balance_amount / long_price) * self.trading_fee_multiplier
        self.long_price = long_price
        self.balance_unit = symbol
        self.buys.append([symbol, time, long_price, 'long'])
        self.number_of_buys = len(self.buys)
        self.sl = long_price - (env.slm*atr)
        print('\n\n', 'filled long', long_price, time)

    def buy_short(self, symbol, short_price, atr, time):
        self.balance_amount = self.balance_amount * self.trading_fee_multiplier
        self.short_amount = (self.balance_amount / short_price) 
        self.short_price = short_price
        self.short_unit = symbol
        self.buys.append([symbol, time, short_price, 'short'])
        self.number_of_buys = len(self.buys)
        self.sl = short_price + (env.slm*atr)
        print('\n\n', 'filled short', short_price, time)

    def sell_long(self, sell_price, time):
        self.balance_amount = (self.balance_amount * sell_price) * self.trading_fee_multiplier
        self.sells.append([symbol, time, sell_price])
        self.balance_unit = 'BUSD'
    
    def sell_short(self, sell_price, time):
        self.balance_amount = (self.balance_amount + (self.short_amount * ((self.short_price*self.trading_fee_multiplier) - sell_price)))
        self.sells.append([symbol, time, sell_price])
        self.short_unit = None
        self.short_amount = 0
    

    def reset_crosses(self, symbols):
        for symbol in self.symbols:
            self.cross_up[symbol] = False
            self.cross_down[symbol] = False
            self.sma_cross_up[symbol] = False
            self.sma_cross_down[symbol] = False
    
    def good_buy(self, type, bbhigh, bblow, close):
        bbrange = bbhigh - bblow
        maximum = 0
        if type == 'long':
            maximum = bblow + (bbrange * self.max_inner_buy)
        
        if type == 'short':
            maximum = bbhigh - (bbrange * self.max_inner_buy)
        
        return maximum



In [634]:
"""
BUY:
    Long: 
        1. Check if symbol crosses under lower boll. band
        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long

    Short:
        1. Check if symbol crosses over upper boll. band
        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short

SELL:
    Long:
        1. Check if symbol crosses over upper boll. band
        2. Check if close is under stop loss; if so, sell
        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long
    
    Short:
        1. Check if symbol crosses under lower boll. band
        2. Check if close goes over stop loss; if so, sell
        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short
"""

'\nBUY:\n    Long: \n        1. Check if symbol crosses under lower boll. band\n        2. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Buy long\n\n    Short:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Buy short\n\nSELL:\n    Long:\n        1. Check if symbol crosses over upper boll. band\n        2. Check if close is under stop loss; if so, sell\n        3. Check if it goes back under (upper - ((upper-lower)*multiplier)) -> Sell long\n    \n    Short:\n        1. Check if symbol crosses under lower boll. band\n        2. Check if close goes over stop loss; if so, sell\n        3. Check if it goes back over (lower + ((upper-lower)*multiplier)) -> Sell short\n'

In [635]:
env = TradingEnv(balance_amount=100, balance_unit='BUSD', symbols=symbols)
for i in range(len(df)):    
    print(env.balance_amount, env.balance_unit, env.number_of_buys)
    if env.balance_unit == 'BUSD' and env.short_unit == None:  # Want to buy
        for symbol in symbols:

            ######## LONG           (lower + ((upper-lower)*multiplier))
            if env.cross_down[symbol] == True and df[f'{symbol}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)) and (df[f'{symbol}-USD_Close'].iloc[i] <= env.good_buy('long', df[f'{symbol}_Upper_Band'].iloc[i], df[f'{symbol}_Lower_Band'].iloc[i], df[f'{symbol}-USD_Close'].iloc[i])): #and df[f'{symbol}_SuperDir'].iloc[i] == 1 and df[f'{symbol}_RSI'].iloc[i] <= 30
                env.buy_long(symbol, long_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i],time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_Low'].iloc[i] <= df[f'{symbol}_Lower_Band'].iloc[i]:
                env.cross_down[symbol] = True
    

            ######## SHORT
            if env.cross_up[symbol] == True and df[f'{symbol}-USD_Close'].iloc[i] <= (df[f'{symbol}_Upper_Band'].iloc[i] - ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)) and (df[f'{symbol}-USD_Close'].iloc[i] >= env.good_buy('short', df[f'{symbol}_Upper_Band'].iloc[i], df[f'{symbol}_Lower_Band'].iloc[i], df[f'{symbol}-USD_Close'].iloc[i])): #and df[f'{symbol}_SuperDir'].iloc[i] == -1 and df[f'{symbol}_RSI'].iloc[i] >= 70
                env.buy_short(symbol, short_price=df[f'{symbol}-USD_Close'].iloc[i], atr=df[f'{symbol}_ATR'].iloc[i], time=df['OpenTime'].iloc[i])
                env.reset_crosses(symbols)
                break
            
            if df[f'{symbol}-USD_High'].iloc[i] >= df[f'{symbol}_Upper_Band'].iloc[i]:
                env.cross_up[symbol] = True



    ####################################################################################################################

    ############## Want to sell long position
    elif env.balance_unit != 'BUSD':

        # Sell long for profit
        if env.cross_up[env.balance_unit] == True and df[f'{env.balance_unit}-USD_Close'].iloc[i] <= (df[f'{symbol}_Upper_Band'].iloc[i] - ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at stop loss
        elif df[f'{env.balance_unit}-USD_Close'].iloc[i] <= env.sl:
            print('Stop loss hit: sell long', df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_long(sell_price=df[f'{env.balance_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell long at break-even  
        if (env.sma_cross_up[env.balance_unit] == True) and (df[f'{env.balance_unit}-USD_Low'].iloc[i] <= env.long_price*((1/env.trading_fee_multiplier)**2)) and (df[f'{env.balance_unit}-USD_High'].iloc[i] >= env.long_price*((1/env.trading_fee_multiplier)**2)):
            print('Break even hit: sell long', env.long_price*((1/env.trading_fee_multiplier)**2), df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_long(sell_price=env.long_price*((1/env.trading_fee_multiplier)**2), time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Check if SMA is crossed up
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_SMA'].iloc[i]:
            print('SMA long crossed')
            env.sma_cross_up[env.balance_unit] = True

        # Check if upper band is crossed
        if df[f'{env.balance_unit}-USD_High'].iloc[i] >= df[f'{env.balance_unit}_Upper_Band'].iloc[i]:
            print('long crossed')
            env.cross_up[env.balance_unit] = True
    

    ############## Want to sell short position
    else:
        print(env.short_amount)
        # Sell short for profit
        if env.cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_Close'].iloc[i] >= (df[f'{symbol}_Lower_Band'].iloc[i] + ((df[f'{symbol}_Upper_Band'].iloc[i]-df[f'{symbol}_Lower_Band'].iloc[i]) * env.band_stop_run)):
            print('sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.wins += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at stop loss
        elif df[f'{env.short_unit}-USD_Close'].iloc[i] >= env.sl:
            print('Stop loss hit: sell short', df[f'{env.short_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i], '\n\n')
            env.losses += 1
            env.sell_short(sell_price=df[f'{env.short_unit}-USD_Close'].iloc[i], time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue
        
        # Sell short at break-even  
        if env.sma_cross_down[env.short_unit] == True and df[f'{env.short_unit}-USD_High'].iloc[i] >= env.short_price*(env.trading_fee_multiplier**2) and df[f'{env.short_unit}-USD_Low'].iloc[i] <= env.short_price*(env.trading_fee_multiplier**2):
            print('Break even hit: sell short', env.short_price*((env.trading_fee_multiplier)**2), df['OpenTime'].iloc[i], '\n\n')
            env.ties += 1
            env.sell_short(sell_price=env.short_price*((env.trading_fee_multiplier)**2), time=df['OpenTime'].iloc[i])
            env.reset_crosses(symbols)
            continue

        # Check if SMA is crossed down
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_SMA'].iloc[i]:
            print('SMA short crossed')
            env.sma_cross_down[env.short_unit] = True

        # Check if lower band is crossed
        if df[f'{env.short_unit}-USD_Low'].iloc[i] <= df[f'{env.short_unit}_Lower_Band'].iloc[i]:
            print('short crossed')
            env.cross_down[env.short_unit] = True





if env.balance_unit != 'BUSD':
    env.sell_long(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
if env.short_unit != None:
    env.sell_short(df[f'{env.short_unit}-USD_Close'].iloc[-1], df['CloseTime'].iloc[-1])
print(env.balance_amount, env.balance_unit, env.number_of_buys, env.wins, env.ties, env.losses)

100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0
100 BUSD 0


 filled long 29057.48 2021-01-01 03:00:00
0.0034400780797233625 BTC 1
0.0034400780797233625 BTC 1
0.0034400780797233625 BTC 1
0.0034400780797233625 BTC 1
SMA long crossed
0.0034400780797233625 BTC 1
SMA long crossed
0.0034400780797233625 BTC 1
SMA long crossed
long crossed
0.0034400780797233625 BTC 1
sell long 29257.84 2021-01-01 04:45:00 


100.60899434243576 BUSD 1
100.60899434243576 BUSD 1
100.60899434243576 BUSD 1


 filled short 29329.71 2021-01-01 05:30:00
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
100.56875074469879 BUSD 2
0.003428903686558742
1

In [636]:

data = pd.DataFrame([(i[1] for i in env.buys), (j[2] for j in env.buys), (a[3] for a in env.buys), (k[1] for k in env.sells), (l[2] for l in env.sells)])
data

,0,1,2,3,4,5,6,7,8,9,...,1433,1434,1435,1436,1437,1438,1439,1440,1441,1442
0,2021-01-01 03:00:00,2021-01-01 05:30:00,2021-01-01 09:15:00,2021-01-01 12:30:00,2021-01-01 20:00:00,2021-01-01 21:15:00,2021-01-02 01:30:00,2021-01-02 09:15:00,2021-01-02 16:15:00,2021-01-03 01:15:00,...,2021-12-29 05:00:00,2021-12-29 08:30:00,2021-12-29 19:15:00,2021-12-30 01:45:00,2021-12-30 07:45:00,2021-12-30 12:30:00,2021-12-30 16:15:00,2021-12-30 21:30:00,2021-12-31 04:30:00,2021-12-31 13:15:00
1,29057.48,29329.71,29504.02,29240.75,29184.47,29496.21,29681.76,31148.23,31652.19,33881.57,...,47920.07,46898.9,46499.76,46903.55,47522.12,47614.83,47308.01,46891.32,48276.48,46940.18
2,long,short,short,long,long,short,short,short,long,short,...,short,long,long,short,short,short,long,long,short,long
3,2021-01-01 04:45:00,2021-01-01 08:00:00,2021-01-01 10:45:00,2021-01-01 15:15:00,2021-01-01 20:45:00,2021-01-02 00:00:00,2021-01-02 04:45:00,2021-01-02 13:45:00,2021-01-03 00:45:00,2021-01-03 04:30:00,...,2021-12-29 08:00:00,2021-12-29 18:15:00,2021-12-29 22:45:00,2021-12-30 04:00:00,2021-12-30 10:15:00,2021-12-30 15:00:00,2021-12-30 20:30:00,2021-12-30 23:45:00,2021-12-31 11:00:00,2021-12-31 19:14:59.999
4,29257.84,29306.250925,29256.46,29264.156643,29441.76,29472.617751,29658.019341,33044.52,33781.84,33854.470165,...,46891.94,46936.441643,46536.98214,46866.034665,47484.109908,47576.745754,47345.879128,47335.73,48014.77,46340.62
